#  Downloading Audio
## Anna Bernbaum
## April 2019

Problems with using the Google AudioSet TFRecords:
- Lack of understanding what '128 - dimension audio features extracted at 1 Hz' actually means
- Difference between frame level features and video level features is unclear
- Embedding? Can you un-embed something?

New tactic:
- Download the actual audio clips for all speech and cough samples from youtube
- Create a spectrogram (or mel spectrogram) for each 
- Train the model on these images

In [1]:
import pandas as pd
import youtube_dl
from pydub import AudioSegment
from pydub.utils import which
from pydub.utils import mediainfo
import librosa as librosa
import os
from random import shuffle
import csv

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [20, 5]

# Get all desired YouTubeIDs

Let's find the index and code associated with 'Cough':

In [2]:
cough_class_label_index = !grep Cough AudioSet/class_labels_indices.csv
print(cough_class_label_index)

speech_class_label_index = !grep Speech AudioSet/class_labels_indices.csv
print(speech_class_label_index)


['47,/m/01b_21,"Cough"']
['0,/m/09x0r,"Speech"', '7,/m/0brhx,"Speech synthesizer"']


Retreiving the class label:

In [3]:
print("Cough:", cough_class_label_index[0].split(",")[1])
print("Speech:", speech_class_label_index[0].split(",")[1])

Cough: /m/01b_21
Speech: /m/09x0r


Finding all samples with the selected label. The header of this table is printed.

In [4]:
# coughs = !grep /m/01b_21 AudioSet/balanced_train_segments.csv |head  # ID manually inserted
# print(type(coughs))
# print(coughs)

# Maybe select speech clips with only a speech tag
# ignore clips with speech and cough tag

coughs_bal = !grep /m/01b_21 AudioSet/balanced_train_segments.csv | cut -c -11
coughs_bal_starts = !grep /m/01b_21 AudioSet/balanced_train_segments.csv | cut -d ',' -f 2
print("Cough samples in Balanced Train:", len(coughs_bal))

speech_bal = !grep /m/09x0r AudioSet/balanced_train_segments.csv | cut -c -11
speech_bal_starts = !grep /m/09x0r AudioSet/balanced_train_segments.csv | cut -d ',' -f 2
print("Speech samples in Balanced Train:", len(speech_bal))


coughs_eval = !grep /m/01b_21 AudioSet/eval_segments.csv | cut -c -11
coughs_eval_starts = !grep /m/01b_21 AudioSet/eval_segments.csv | cut -d ',' -f 2
print("\nCough samples in Evaluation:", len(coughs_eval))

speech_eval = !grep /m/09x0r AudioSet/eval_segments.csv | cut -c -11
speech_eval_starts = !grep /m/09x0r AudioSet/eval_segments.csv | cut -d ',' -f 2
print("Speech samples in Evaluation:", len(speech_eval))


coughs_unbal = !grep /m/01b_21 AudioSet/unbalanced_train_segments.csv | cut -c -11
coughs_unbal_starts = !grep /m/01b_21 AudioSet/unbalanced_train_segments.csv | cut -d ',' -f 2
print("\nCough samples in Unbalanced Train:", len(coughs_unbal))

speech_unbal = !grep /m/09x0r AudioSet/unbalanced_train_segments.csv | cut -c -11
speech_unbal_starts = !grep /m/09x0r AudioSet/unbalanced_train_segments.csv | cut -d ',' -f 2
print("Speech samples in Unbalanced Train:", len(speech_unbal))
      
print("\nTotal Number of Cough Clips:", (len(coughs_bal)+len(coughs_eval)+len(coughs_unbal)))
print("\nTotal Number of Speech Clips:", (len(speech_bal)+len(speech_eval)+len(speech_unbal)))

Cough samples in Balanced Train: 60
Speech samples in Balanced Train: 5735

Cough samples in Evaluation: 60
Speech samples in Evaluation: 5324

Cough samples in Unbalanced Train: 751
Speech samples in Unbalanced Train: 999421

Total Number of Cough Clips: 871

Total Number of Speech Clips: 1010480


Let's create our own Train : Evaluation : Validation split to maximise the number of clips used. This will be done in a separate file, but here we combine all available clips.

In [5]:
# combine the lists of youtube ids into one master list
all_coughs = (coughs_bal + coughs_eval + coughs_unbal)
all_speech = (speech_bal + speech_eval + speech_unbal)

all_coughs_starts = (coughs_bal_starts + coughs_eval_starts + coughs_unbal_starts)
all_speech_starts = (speech_bal_starts + speech_eval_starts + speech_unbal_starts)


# create pandas dataframe of YouTubeIDs
speech_df = pd.DataFrame()  # create empty dataframe
cough_df = pd.DataFrame()
speech_df['Speech YouTubeIDs'] = pd.Series(all_speech)
cough_df['Coughs YouTubeIDs'] = pd.Series(all_coughs)
speech_df['Speech Starts'] = pd.Series(all_speech_starts)
cough_df['Coughs Starts'] = pd.Series(all_coughs_starts)

# Shuffle the dataframes
speech_df = speech_df.sample(frac=1).reset_index(drop=True)
cough_df = cough_df.sample(frac=1).reset_index(drop=True)

# Trim Speech Df
speech_df = speech_df[:871]

print(speech_df)
print(cough_df)

    Speech YouTubeIDs Speech Starts
0         QVUn9VPburE         8.000
1         BmaJagrZoeM        50.000
2         Cf02eu2D_Hs        30.000
3         t8jUR81_Qbw        30.000
4         obahqOGbA4U       240.000
5         QAuLVVjN3cI        18.000
6         ATkiruac-y4        30.000
7         6q3FwDLGA0g       170.000
8         z67ie5IhMtU        30.000
9         cTc9kM0oN3Y        30.000
10        h1QxG72W3g0       430.000
11        x-mgo5sWa4E       120.000
12        SC-MCEqtSHc        30.000
13        A39QbQ3Z2qk        30.000
14        4E0Kicq90NM        30.000
15        1jQW69fxTW0        27.000
16        Icv0XlAuBGo       370.000
17        vsPf6n0HSA8        30.000
18        AYaN9NqpVA0        30.000
19        ogDl7PHyyhE       160.000
20        bJjYBAwglxM       210.000
21        YHfe-pFhfT4        30.000
22        3IAQtwMWMwM       430.000
23        NkwMxUW-Kds       170.000
24        4wEqOLNZiJU        30.000
25        GwXzQvqmgQg        17.000
26        7R6Hzrp1Ul8       

# Find Associated YouTube clip for each YouTubeID

https://stackoverflow.com/questions/27473526/download-only-audio-from-youtube-video-using-youtube-dl-in-python-script

https://stackoverflow.com/questions/28423501/download-part-of-the-youtube-video-using-python


## Getting all Speech Clips

In [6]:
speech_test_df = speech_df[:50]

speech_fail_log = []

for i in range(len(speech_test_df.index)):
    title = speech_test_df['Speech YouTubeIDs'][i] + '.wav'  # filename
    url = "http://www.youtube.com/watch?v=" + title  # URL

    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': 'Untrimmed_AudioSet_WAV_files/' + title,
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',  # WAV file
            'preferredquality': '192',
        }],
    }
    
    try:
        with youtube_dl.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])

        # Find the associated start_time
        start_time = float(speech_test_df['Speech Starts'][i])

        # Trim the clip
        y, sr = librosa.load('Untrimmed_AudioSet_WAV_files/'+ title, offset=start_time, duration=10.0) # trim a 10 second segment from start_time    

    #     # plot the clip - used for debugging
    #     plt.title(title)
    #     plt.xlabel('sample number')
    #     plt.ylabel('Amplitude')
    #     plt.plot(y)
    #     plt.show()

        # write a wav file of the trimmed clip
        filepath = 'Audioset_WAV_files/Speech/' + title
        librosa.output.write_wav(filepath, y, sr, norm=True)
        librosa.output.write_wav(filepath, y, sr, norm=True)

        
    # if the clip is unavailable
    except: 
        speech_fail_log.append(speech_test_df['Speech YouTubeIDs'][i])

        
print(speech_fail_log)

[youtube] QVUn9VPburE: Downloading webpage
[youtube] QVUn9VPburE: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/QVUn9VPburE.wav
[download] 100% of 206.31KiB in 00:0045MiB/s ETA 00:000
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/QVUn9VPburE.wav exists, skipping
[youtube] BmaJagrZoeM: Downloading webpage
[youtube] BmaJagrZoeM: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/BmaJagrZoeM.wav
[download] 100% of 2.29MiB in 00:0232MiB/s ETA 00:002
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/BmaJagrZoeM.wav exists, skipping
[youtube] Cf02eu2D_Hs: Downloading webpage
[youtube] Cf02eu2D_Hs: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/Cf02eu2D_Hs.wav
[download] 100% of 3.35MiB in 00:0101MiB/s ETA 00:004
[ffmpeg] Correcting container in "Untrimmed_AudioSet_WAV_files/Cf02eu2D_Hs.wav"
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/Cf02eu2D_Hs.wav exists, 

ERROR: This video is no longer available because the YouTube account associated with this video has been terminated.


[youtube] obahqOGbA4U: Downloading webpage
[youtube] obahqOGbA4U: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/obahqOGbA4U.wav
[download] 100% of 5.53MiB in 00:0162MiB/s ETA 00:009
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/obahqOGbA4U.wav exists, skipping
[youtube] QAuLVVjN3cI: Downloading webpage
[youtube] QAuLVVjN3cI: Downloading video info webpage
[youtube] QAuLVVjN3cI: Downloading MPD manifest
[dashsegments] Total fragments: 4
[download] Destination: Untrimmed_AudioSet_WAV_files/QAuLVVjN3cI.wav
[download] 100% of 331.00KiB in 00:03.35MiB/s ETA 00:001
[ffmpeg] Correcting container in "Untrimmed_AudioSet_WAV_files/QAuLVVjN3cI.wav"
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/QAuLVVjN3cI.wav exists, skipping
[youtube] ATkiruac-y4: Downloading webpage
[youtube] ATkiruac-y4: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/ATkiruac-y4.wav
[download] 100% of 4.85MiB in 00:0327MiB/s ETA 00:0

ERROR: This video is unavailable.


[youtube] 4E0Kicq90NM: Downloading webpage
[youtube] 4E0Kicq90NM: Downloading video info webpage
[youtube] 4E0Kicq90NM: Downloading MPD manifest
[dashsegments] Total fragments: 11
[download] Destination: Untrimmed_AudioSet_WAV_files/4E0Kicq90NM.wav
[download] 100% of 1.04MiB in 00:02.20MiB/s ETA 00:000012
[ffmpeg] Correcting container in "Untrimmed_AudioSet_WAV_files/4E0Kicq90NM.wav"
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/4E0Kicq90NM.wav exists, skipping
[youtube] 1jQW69fxTW0: Downloading webpage
[youtube] 1jQW69fxTW0: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/1jQW69fxTW0.wav
[download] 100% of 574.34KiB in 00:0044MiB/s ETA 00:000
[ffmpeg] Correcting container in "Untrimmed_AudioSet_WAV_files/1jQW69fxTW0.wav"
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/1jQW69fxTW0.wav exists, skipping
[youtube] Icv0XlAuBGo: Downloading webpage
[youtube] Icv0XlAuBGo: Downloading video info webpage
[download] Destination: Untrimmed_Au

[dashsegments] Total fragments: 14
[download] Destination: Untrimmed_AudioSet_WAV_files/1w979JoVyXM.wav
[download] 100% of 1.91MiB in 00:08.11MiB/s ETA 00:00051
[ffmpeg] Correcting container in "Untrimmed_AudioSet_WAV_files/1w979JoVyXM.wav"
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/1w979JoVyXM.wav exists, skipping
[youtube] 7jvf2fwhkXY: Downloading webpage
[youtube] 7jvf2fwhkXY: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/7jvf2fwhkXY.wav
[download] 100% of 3.71MiB in 00:0183MiB/s ETA 00:004
[ffmpeg] Correcting container in "Untrimmed_AudioSet_WAV_files/7jvf2fwhkXY.wav"
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/7jvf2fwhkXY.wav exists, skipping
[youtube] 5l8Y4twyb6c: Downloading webpage
[youtube] 5l8Y4twyb6c: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/5l8Y4twyb6c.wav
[download] 100% of 737.10KiB in 00:0013MiB/s ETA 00:000
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/5l8

## Getting all cough clips

In [7]:
coughs_test_df = cough_df[:50]

coughs_fail_log = []

for i in range(len(coughs_test_df.index)):
    title = coughs_test_df['Coughs YouTubeIDs'][i] + '.wav'  # filename
    url = "http://www.youtube.com/watch?v=" + title  # URL

    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': 'Untrimmed_AudioSet_WAV_files/' + title,
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',  # WAV file
            'preferredquality': '192',
        }],
    }
    
    # if clip is unavailable
    # if clip is longer than X skip
    try:
        with youtube_dl.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
        # Find the associated start_time
        start_time = float(coughs_test_df['Coughs Starts'][i])


        # Trim the clip
        y, sr = librosa.load('Untrimmed_AudioSet_WAV_files/' + title, offset=start_time, duration=10.0) # trim a 10 second segment from start_time    

    #     # plot the clip - used for debugging
    #     plt.title(title)
    #     plt.xlabel('sample number')
    #     plt.ylabel('Amplitude')
    #     plt.plot(y)
    #     plt.show()

        # write over orginal file with the 10 second clip
        filepath = 'Audioset_WAV_files/Coughs/' + title
        librosa.output.write_wav(filepath, y, sr, norm=True)
        librosa.output.write_wav(filepath, y, sr, norm=True)

    # if clip is unavailable
    except: 
        coughs_fail_log.append(coughs_test_df['Coughs YouTubeIDs'][i])
        # create an error log

print(coughs_fail_log)

[youtube] O61IKTdt2I0: Downloading webpage
[youtube] O61IKTdt2I0: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/O61IKTdt2I0.wav
[download] 100% of 483.19KiB in 00:0003MiB/s ETA 00:000
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/O61IKTdt2I0.wav exists, skipping
[youtube] RYppdW56lfo: Downloading webpage
[youtube] RYppdW56lfo: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/RYppdW56lfo.wav
[download] 100% of 1.73MiB in 00:0073MiB/s ETA 00:002
[ffmpeg] Correcting container in "Untrimmed_AudioSet_WAV_files/RYppdW56lfo.wav"
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/RYppdW56lfo.wav exists, skipping
[youtube] quwLxxbH2Pg: Downloading webpage
[youtube] quwLxxbH2Pg: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/quwLxxbH2Pg.wav
[download] 100% of 545.97KiB in 00:0079MiB/s ETA 00:000
[ffmpeg] Correcting container in "Untrimmed_AudioSet_WAV_files/quwLxxbH2Pg.wav

[youtube] xCRumdDLFj0: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/xCRumdDLFj0.wav
[download] 100% of 1.70MiB in 00:0117MiB/s ETA 00:001
[ffmpeg] Correcting container in "Untrimmed_AudioSet_WAV_files/xCRumdDLFj0.wav"
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/xCRumdDLFj0.wav exists, skipping
[youtube] 3id3zRRZBVM: Downloading webpage
[youtube] 3id3zRRZBVM: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/3id3zRRZBVM.wav
[download] 100% of 2.07MiB in 00:02.78KiB/s ETA 00:00
[ffmpeg] Correcting container in "Untrimmed_AudioSet_WAV_files/3id3zRRZBVM.wav"
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/3id3zRRZBVM.wav exists, skipping
[youtube] VzVjieAgz7Y: Downloading webpage
[youtube] VzVjieAgz7Y: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/VzVjieAgz7Y.wav
[download] 100% of 1.74MiB in 00:0038MiB/s ETA 00:002
[ffmpeg] Post-process file Untrimmed_AudioSet

ERROR: This video is unavailable.


[youtube] cl5Bt-rqtZ4: Downloading webpage
[youtube] cl5Bt-rqtZ4: Downloading video info webpage
[youtube] cl5Bt-rqtZ4: Downloading MPD manifest
[dashsegments] Total fragments: 8
[download] Destination: Untrimmed_AudioSet_WAV_files/cl5Bt-rqtZ4.wav
[download] 100% of 799.38KiB in 00:01.33MiB/s ETA 00:000
[ffmpeg] Correcting container in "Untrimmed_AudioSet_WAV_files/cl5Bt-rqtZ4.wav"
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/cl5Bt-rqtZ4.wav exists, skipping
[youtube] jMyY06bjVRA: Downloading webpage
[youtube] jMyY06bjVRA: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/jMyY06bjVRA.wav
[download] 100% of 2.09MiB in 00:0051MiB/s ETA 00:002
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/jMyY06bjVRA.wav exists, skipping
[youtube] 89ZJ46zuxRY: Downloading webpage
[youtube] 89ZJ46zuxRY: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/89ZJ46zuxRY.wav
[download] 100% of 1.10MiB in 00:0071MiB/s ETA 00:0

ERROR: This video has been removed for violating YouTube's Terms of Service.


[youtube] s77vdz6Xnj8: Downloading webpage
[youtube] s77vdz6Xnj8: Downloading video info webpage


ERROR: This video is unavailable.


[youtube] 6iz8Jk8Hvwg: Downloading webpage
[youtube] 6iz8Jk8Hvwg: Downloading video info webpage


ERROR: This video has been removed by the user


[youtube] 6nTcsNoIGDw: Downloading webpage
[youtube] 6nTcsNoIGDw: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/6nTcsNoIGDw.wav
[download] 100% of 1.33MiB in 00:0157MiB/s ETA 00:000
[ffmpeg] Correcting container in "Untrimmed_AudioSet_WAV_files/6nTcsNoIGDw.wav"
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/6nTcsNoIGDw.wav exists, skipping
[youtube] RWCSzp1zU8A: Downloading webpage
[youtube] RWCSzp1zU8A: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/RWCSzp1zU8A.wav
[download] 100% of 1.68MiB in 00:0162MiB/s ETA 00:001
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/RWCSzp1zU8A.wav exists, skipping
[youtube] qOSdHmfwLF4: Downloading webpage
[youtube] qOSdHmfwLF4: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/qOSdHmfwLF4.wav
[download] 100% of 2.14MiB in 00:0181MiB/s ETA 00:002
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/qOSdHmfwLF4.wav exists, sk

## Check number of clips after downloading
Some clips were lost due to failed downloads. The two datagroups need to be evened out.

In [8]:
# Check number of clips in each folder
coughs_count=0
for files in os.listdir("Audioset_WAV_files/Coughs/"):
    if files.endswith('.wav'):
        coughs_count+=1
print("Unedited number of cough clips:", coughs_count)


speech_count=0
for files in os.listdir("Audioset_WAV_files/Speech/"):
    if files.endswith('.wav'):
        speech_count+=1
print("Unedited number of speech clips:", speech_count)

# Remove additional clips to even out the size
values = {"Coughs": coughs_count, "Speech": speech_count}
largest = max(values, key=values.get)  # find which group has more clips
difference = (abs(coughs_count - speech_count))  # how many clips should be removed

# find names of clips to be removed
f = []
for (dirpath, dirnames, filenames) in os.walk("Audioset_WAV_files/"+ largest + '/'):
    f.extend(filenames)
    break

shuffle(f)

clips_to_remove = f[:difference]
print(clips_to_remove)

for clip in clips_to_remove:
    os.remove("Audioset_WAV_files/"+ largest + '/' + clip)

# Check it has worked
coughs_count=0
for files in os.listdir("Audioset_WAV_files/Coughs/"):
    if files.endswith('.wav'):
        coughs_count+=1
print("Unedited number of cough clips:", coughs_count)


speech_count=0
for files in os.listdir("Audioset_WAV_files/Speech/"):
    if files.endswith('.wav'):
        speech_count+=1
print("Unedited number of speech clips:", speech_count)

Unedited number of cough clips: 46
Unedited number of speech clips: 48
['ATkiruac-y4.wav', 'o80gIVpvsPM.wav']
Unedited number of cough clips: 46
Unedited number of speech clips: 46


# Save YouTube IDs of final clip selections

In [9]:
# get filename of every file in both directories
# write to CSV  selected_YouTubeIDs

coughs_clips = ["Coughs"]
for files in os.listdir("Audioset_WAV_files/Coughs/"):
    if files.endswith('.wav'):
        coughs_clips.append(files)
        print(files)

speech_clips = ["Speech"]
for files in os.listdir("Audioset_WAV_files/Speech/"):
    if files.endswith('.wav'):
        speech_clips.append(files)
        print(files)

csvData = zip(coughs_clips, speech_clips)      
with open('selected_YouTubeIDs.csv', 'w') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerows(csvData)

3Liy9uBgsQM.wav
xk57B6zi6hA.wav
txVBnPl9KPM.wav
GXLEKU6K1uI.wav
GzdmC_MiIyg.wav
qHxgEpRG1Vs.wav
BR6UqPUqYsQ.wav
_0WKVY0n8aE.wav
k_0J26cnYpw.wav
bv35U83Ob1o.wav
8ieJbzu7ql8.wav
RYppdW56lfo.wav
VlFNQv6fLqQ.wav
qj-s6ZcgytA.wav
HEKKh5yZ1s8.wav
E5guDHgn7XQ.wav
Z459PzdNuCU.wav
tUUkucw-BOY.wav
jMyY06bjVRA.wav
89ZJ46zuxRY.wav
vA-eGyCdVBE.wav
quwLxxbH2Pg.wav
qOSdHmfwLF4.wav
dFmT9hZ5sbA.wav
oGf-eDCiQfg.wav
-vu4jJkffMw.wav
nL3LUxrWEpA.wav
4txOUgXllWE.wav
EksYlo1IXU0.wav
3id3zRRZBVM.wav
1MSYO4wgiag.wav
aJdyPN00-bM.wav
X8yUSV4oqoU.wav
RwE9JAktTvU.wav
O61IKTdt2I0.wav
RWCSzp1zU8A.wav
TA-iHSeEUYk.wav
6nTcsNoIGDw.wav
Ao0n1cqfFDw.wav
sNCKIJFUj5I.wav
VzVjieAgz7Y.wav
sUFGh7zp9D0.wav
9KNqsONT3-Y.wav
cl5Bt-rqtZ4.wav
o-TJISpYLFc.wav
xCRumdDLFj0.wav
5l8Y4twyb6c.wav
GwXzQvqmgQg.wav
OG_aj0e2gCg.wav
ogDl7PHyyhE.wav
0sLzEXwCV50.wav
h1QxG72W3g0.wav
brsMKP0yP8I.wav
uFMH9z6DMIE.wav
Cf02eu2D_Hs.wav
1w979JoVyXM.wav
4NtspP5IbfI.wav
DVhN2nJSoi0.wav
V44Qc-TpuIE.wav
6KL1ZzMv7fw.wav
Icv0XlAuBGo.wav
NkwMxUW-Kds.wav
Cma9PIQj